In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/feedback-prize-english-language-learning/sample_submission.csv
/kaggle/input/feedback-prize-english-language-learning/train.csv
/kaggle/input/feedback-prize-english-language-learning/test.csv


In [2]:
import pandas as pd
from transformers import AutoTokenizer,AutoModel,AutoConfig
from transformers import get_polynomial_decay_schedule_with_warmup
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
!nvidia-smi


Mon Mar 10 13:28:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P0             26W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
base_dir="/kaggle/working/"
df_training_path="../input/feedback-prize-english-language-learning/train.csv"
df_test_path="../input/feedback-prize-english-language-learning/train.csv"
sample_submission_path="../input/feedback-prize-english-language-learning/sample_submission.csv"


In [5]:
config={}
config['base_dir']=base_dir

In [6]:
df=pd.read_csv(df_training_path)

In [7]:
df.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [8]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['cohesion'])
train_df=train_df.reset_index(drop=True)
val_df=val_df.reset_index(drop=True)


In [9]:
val_df.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,F6DEF203DCD1,First impressions is what the person show you....,3.5,3.5,3.5,3.0,3.0,2.5
1,4F75DF4CEBE9,This topic brings in a lot of question and con...,2.5,3.0,3.0,2.0,2.0,2.5
2,F069420C6DB7,Some schools are offering distance learning fo...,3.5,3.0,3.0,3.5,4.0,3.0
3,E897534557AF,Recently technology have been a great impact i...,2.5,3.0,3.0,3.0,3.0,2.5
4,83F57096AF9C,Be a master in any area is something really aw...,3.5,3.0,3.5,3.0,2.5,3.0


In [ ]:
config['tokenizer']=AutoTokenizer.from_pretrained("microsoft/deberta-v3-large")
def tokenize(df):

    text=df["full_text"]

    tokenized=tokenizer(text,add_special_tokens=True,max_length=512,padding=True,truncation=True)
    tokenized["label"]=[df[i] for i in config["tcols"]]
    tokenized["length"]=len(tokenized["input_ids"])

    return tokenized
    

In [ ]:
df_test=pd.read_csv(df_test_path)
config['tcols']=[col for col in df.columns if col not in ['text_id','full_text']]
print(config['tcols'])

In [ ]:
df_test[config["tcols"]]=0
df_test.head()

In [ ]:
from datasets import Dataset

ds=Dataset.from_pandas(df_test)
ds

In [ ]:
import torch.nn as nn
import torch

class MeanPooling:
    def meanpooling(self,last_hidden_state,attention_mask):
        attention_mask=attention_mask.unsqueeze(-1).expand_as(last_hidden_state)
        pooled=torch.sum(attention_mask*last_hidden_state,dim=1)
        token_count=torch.sum(attention_mask,dim=1)
        token_count=torch.clamp(token_count,min=1e-5)
        pooled=pooled/token_count
        return pooled

In [ ]:
config['max_length']=512
config['device']=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch
from torch.utils.data import Dataset

def prepare_input(text):
    if not isinstance(text, str):  # Ensure text is a string
        raise ValueError(f"Expected to get a string, but got {type(text)}")
    
    inputs = config['tokenizer'].encode_plus(
        text, 
        return_tensors=None, 
        add_special_tokens=True, 
        max_length=config['max_length'],
        padding='max_length',  # Use 'padding' instead of 'pad_to_max_length'
        truncation=True
    )
    
    return {k: torch.tensor(v, dtype=torch.long) for k, v in inputs.items()}


class FeedbackPrizeDataset(Dataset):
    def __init__(self, df):
        self.texts = df['full_text'].values
        self.labels = df[config["tcols"]].values  # Fixed missing bracket
    
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.texts[item])  # Removed self.cfg
        label = torch.tensor(self.labels[item], dtype=torch.float)
        return {**inputs, "label":label}
    

def collate(inputs):
    mask_len = int(inputs["attention_mask"].sum(axis=1).max())
    for k, v in inputs.items():
        inputs[k] = inputs[k][:, :mask_len]
    return inputs 


In [ ]:
config['encoder_lr']=1e-5
config['decoder_lr']=1e-9
config['weight_decay']=1e-4
config['batch_size']=8
config['n_accumulation']=1
config['epochs']=1
config['train_bs']=3
config['test_bs']=3
config['lr']=1e-5

In [ ]:
class DebertaModel(nn.Module):
    def __init__(self,config):
        super().__init__()
        # self.config=config

        self.model_config=AutoConfig.from_pretrained("microsoft/deberta-v3-large")
        self.config=config
        
        update_config={
                "output_hidden_states": True,
                "hidden_dropout_prob": 0.,
                "add_pooling_layer": False,
                "attention_probs_dropout_prob":0,
            }

        for key,val in update_config.items():
            if hasattr(self.model_config,key):
                setattr(self.model_config,key,val)

        self.model=AutoModel.from_pretrained("microsoft/deberta-v3-large",config=self.model_config)
        self.layernorm=nn.LayerNorm(self.model.config.hidden_size*2)
        self.out=nn.Linear(self.model.config.hidden_size*2,len(config["tcols"]))

        self.layer_weight=nn.Parameter(torch.zeros(self.model_config.num_hidden_layers).view(self.model_config.num_hidden_layers,1,1,1))

        self.dropout=nn.ModuleList(
            [nn.Dropout(p=i*0.2) for i in range(5)]
        )

        self.loss=nn.SmoothL1Loss(reduction='mean')

    def forward(self,input_ids,attention_mask):

        output=self.model(input_ids,attention_mask)
        hidden_outputs=output.hidden_states

        hidden_states= torch.stack(tuple(hidden_outputs[-i-1] for i in range(len(hidden_outputs)-1)),dim=0)

        layer_weight=F.softmax(self.layer_weight,dim=0)

        output_mean=torch.mean(layer_weight*hidden_states,dim=0)

        output_max,_=torch.max(hidden_states,dim=0)

        output=torch.cat((output_mean,output_max),dim=-1)

        output=output[:,0,:]

        logits=None
        
        for i ,dropout in enumerate(self.dropout):
            if i==0:
                logits=dropout(self.out(output))
            else:
                logits+=dropout(self.out(output))
            
        return logits/len(self.dropout)

    def get_optimizer_params(self,model,encoder_lr,decoder_lr,weight_decay):


        no_decay_list=["bias","LayerNorm.weight"]
        optimizer_groups=[
            {
                'params':[p for n,p in model.model.named_parameters() if not any( nd in n for nd in no_decay_list)],
                'lr':encoder_lr,
                'weight_decay': weight_decay
            },
            {
                'params':[p for n,p in model.model.named_parameters() if any( nd in n for nd in no_decay_list)],
                'lr':encoder_lr,
                'weight_decay':0.0
            },
            {
                'params':[p for n,p in model.named_parameters() if "model" not in n],
                'lr':decoder_lr,
                'weight_decay':weight_decay
            }
        ]

        return optimizer_groups

    def configure_optimizer(self):

        parameter_groups=self.get_optimizer_params(self,self.config['encoder_lr'],self.config['decoder_lr']
                                                  ,weight_decay=self.config['weight_decay'])

        optimizer=torch.optim.AdamW(parameter_groups,lr=self.config['lr'])

        warmup_steps=0.09*self.config['data_len']//self.config['batch_size']
        training_steps=self.config['epochs']*self.config['data_len']//self.config['batch_size']

        scheduler=get_polynomial_decay_schedule_with_warmup(optimizer,warmup_steps,training_steps,lr_end=7e-7, power=3.0)

        lr_scheduler_config={
            'scheduler':scheduler,
            'interval':'step',
            'frequency':1,
        }

        return {'optimizer':optimizer,'lr_scheduler': scheduler}

In [ ]:
def criterion(output,label):
    return nn.SmoothL1Loss(reduction='mean')(output,label)

In [ ]:
def train_one_epoch(dataloader,model,optimizer,scheduler,device,mode):

    running_loss=0
    average_loss=0
    total_len=0

    if mode=='train':
        model.train()
        torch.set_grad_enabled(True)
    else:
        model.eval()
        torch.set_grad_enabled(False)
    
    for step,data in enumerate(tqdm(dataloader)):
        
        input_ids,attention_mask=data['input_ids'].to(device=device),data['attention_mask'].to(device=device)
        
        output=model(input_ids,attention_mask)

        batch_size=input_ids.size()[0]

        loss=criterion(output,data['label'].to(device=device))
        
        if mode=='train':
            loss.backward()
            
            if (step+1)%config['n_accumulation']==0:
                optimizer.step()
                optimizer.zero_grad()
                
                if scheduler is not None:
                    scheduler.step()

        running_loss+=loss.item()*batch_size
        total_len+=batch_size
        
    average_loss=running_loss/total_len
    return average_loss

In [ ]:
def train_model(config,train_df,val_df):

    train_dataset=FeedbackPrizeDataset(train_df)
    val_dataset=FeedbackPrizeDataset(val_df)

    train_loader=DataLoader(train_dataset,batch_size=config['train_bs'],shuffle=True)
    val_loader=DataLoader(val_dataset,batch_size=config['train_bs'],shuffle=True)

    config['data_len']=len(train_loader)

    model=DebertaModel(config).to(config['device'])
    optimizer_dict=model.configure_optimizer()
    optimizer=optimizer_dict['optimizer']
    lr_scheduler=optimizer_dict['lr_scheduler']

    best_loss=float("inf")
    best_model=model.state_dict()
    best_optimizer_state=optimizer.state_dict()
    
    for epoch in range(config['epochs']):
        train_loss=train_one_epoch(train_loader,model,optimizer,lr_scheduler,config['device'],mode='train')
        val_loss=train_one_epoch(val_loader,model,optimizer,lr_scheduler,config['device'],mode='val')
        if val_loss<best_loss:
            print(f"Loss has improved from {best_loss} to {val_loss} in the epoch:{epoch}")
            best_loss=val_loss
            best_model=model.state_dict()
            best_optimizer_state=optimizer.state_dict()
            
    return best_loss,best_model,best_optimizer_state

In [ ]:
config['test_bs']=1

In [ ]:
import torch
torch.cuda.empty_cache()

best_loss,best_model,_=train_model(config,train_df,val_df)

test_dataset=FeedbackPrizeDataset(df_test)
test_loader=DataLoader(test_dataset,batch_size=config['test_bs'])

model= DebertaModel(config).to(device=config['device'])
model.load_state_dict(best_model)
model.eval()

predictions=[]
with torch.no_grad():
    for data in test_loader:
        input_ids=data['input_ids'].to(device=config['device'])
        attention_mask=data['attention_mask'].to(device=config['device'])

        output=model(input_ids,attention_mask)
        predictions.append(output.cpu().numpy())

    
    predictions=np.concatenate(predictions,axis=0)
    df_submission = pd.DataFrame(predictions, columns=config['tcols'])
    df_submission.insert(0, 'text_id', df_test['text_id'])

    # Load sample submission to ensure format consistency
    sample_submission = pd.read_csv(sample_submission_path)
    df_submission = df_submission[sample_submission.columns]
    
    # Save submission file
    df_submission.to_csv(base_dir + "submission.csv", index=False)
    
    print("Submission file saved at:", base_dir + "submission.csv")
    df_submission.head()